## Seoul Bike Rental | Prediction Deploy | Dash 🚴🏿‍♀️

**needed liberary**

In [1]:
import numpy as np
import pandas as pd
import warnings
import dash
from dash import html
from dash import dcc
from datetime import date
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

import pickle

warnings.filterwarnings('ignore')

In [3]:
# loss function of  model
def RMSLE(y_acual, y_pred):
    return np.sqrt(mean_squared_log_error(y_acual, y_pred))

In [4]:
# load the vectorizer from disk
filename = 'output/Xgb_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [7]:
import sklearn
sklearn.__version__

'0.24.2'

In [5]:
model.predict(np.array([11,1,11,13,23,34,5,2000,3,0,1,1,1,0,0,0,0,0,0,1,0,1,0,0]).reshape(1,-1))

array([1065.9127], dtype=float32)

## Calculate season method

In [4]:
def season_of_date(date_UTC):
    year = str(date_UTC.year)
    seasons = {'spring': pd.date_range(start= year +'-03-21 00:00:00', end=year + '-06-20 00:00:00'),
               'summer': pd.date_range(start= year + '-06-21 00:00:00', end= year + '-09-22 00:00:00'),
               'autumn': pd.date_range(start= year + '-09-23 00:00:00', end= year + '-12-20 00:00:00')}
    if date_UTC in seasons['spring']:
        return 'spring'
    if date_UTC in seasons['summer']:
        return 'summer'
    if date_UTC in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'

## Calculate day, day of week

In [5]:
get_label_day_night = lambda hour : 0 if (hour<7) else(1)

In [6]:
get_working_day = lambda week_day: 0 if (week_day in ['Saturday', 'Sunday']) else(1)

## One hot encode for day_of_week and season

In [7]:
def get_week_day_list(week_day):
    day_dict = {'Friday':0, 'Monday':0, 'Saturday':0,
                'Sunday':0, 'Thursday':0, 'Tuesday':0, 'Wednesday':0}
    day_dict[week_day] = 1
    return [*day_dict.values()]

In [8]:
def get_season_list(season):
    season_dict = {'autumn':0, 'spring':0, 'summer':0, 'winter':0}
    season_dict[season] = 1
    return [*season_dict.values()]

## Prediction method

In [9]:
def prediction(date_value, hour, temp, humidity, wind_speed, visiabilty, solar, rain, holiday, Function_Day):
    date_value = pd.to_datetime(date_value, format="%Y-%m-%d", utc=True)
    day = date_value.day
    month = date_value.month
    week_day = date_value.day_name()
    season = season_of_date(date_value)
    label_day_night = get_label_day_night(hour)
    Working_Day = get_working_day(week_day)
    
    day_list = get_week_day_list(week_day)

    season_list = get_season_list(season)
    
    input_array = np.array([day, label_day_night, month, hour, temp, humidity,
                             wind_speed, visiabilty, solar, rain, holiday, 
                             Function_Day,Working_Day]+day_list+season_list)
    
    my_prediction = np.array([model.predict(input_array.reshape(1,-1))], dtype='int')

    return str(my_prediction[0][0])

In [10]:
# prediction method test
prediction('2021-06-10', 18, 16.6, 61, 2.9, 2000, 0.16, 0,1,1)

'2306'

## Testing method

In [11]:
def input_test(date_value, hour, temp, humidity, wind_speed, visiabilty, solar, rain, holiday, Function_Day):
    
    try:
        if not(all([str(type(x))!="<class 'NoneType'>" for x in [hour, temp, humidity, wind_speed, visiabilty, solar, rain]])):
            Ex = ValueError()
            Ex.strerror = "Make Sure you enter all values."
            raise Ex
            
        if not (type(hour)==int and hour in range(24)):
            Ex = ValueError()
            Ex.strerror = "Hour Value must be within 0 and 23."
            raise Ex
        if not(temp >= -20 and temp <= 50):
            Ex = ValueError()
            Ex.strerror = "Temperature Value should be in range -20 to 50."
            raise Ex
            
        if not(humidity >= 0 and humidity <= 100):
            Ex = ValueError()
            Ex.strerror = "Humidity Value should be precentage in range 0 to 100."
            raise Ex
        
        if not(wind_speed >= 0 and wind_speed <= 10):
            Ex = ValueError()
            Ex.strerror = "Wind Speed Value should be in range 0 to 10."
            raise Ex
            
        if not(visiabilty >= 0 and visiabilty <= 2000):
            Ex = ValueError()
            Ex.strerror = "Visiabilty Value should be in range 0 to 2000."
            raise Ex
        
        if not(solar >= 0 and solar <= 5):
            Ex = ValueError()
            Ex.strerror = "Solar Value should be in range 0 to 5."
            raise Ex
        
        if not(rain >= 0 and rain <= 40):
            Ex = ValueError()
            Ex.strerror = "RainFall Value should be in range 0 to 40."
            raise Ex
        try:    
            output = prediction(date_value, hour, temp, humidity, wind_speed, visiabilty, solar, rain, holiday, Function_Day)
        except:
            Ex = ValueError()
            Ex.strerror = "Model Can't predict these values."
            raise Ex
            
    except ValueError as e:
        output = e.strerror
        
    return output

## Start of Design

In [12]:
# Text field
def drawText():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2("Predict Rental in Seoul"),
                ], style={'textAlign': 'center'}) 
            ])
        ),
    ])

In [13]:
# list field
def drawlist(label_dict,text):
    return dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label(text),
                dbc.Select(
                    id=text,
                    options=label_dict,
                    value=[*label_dict[-1].values()][-1],
                    style={'color':'#aaa',}
                ),
                html.Br(),
            ],)

        ], width=6),
    ], justify="center", align='center')

In [14]:
# input field
def drawInput(input_id, text):
    return dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label(text.split("(")[0]),
                dbc.Input(id=input_id, placeholder=text, type="number", size="md", style={'color':'#fff'}),
                html.Br()
            ]),

        ], width=6),
    ], justify="center", align='center')

In [15]:
# button field
def drawButton(id_value, value):
    return html.Div([
        dbc.Button(id=id_value, children=value, color="secondary", type='submit', size='md', n_clicks=0),
        html.Br()
    ], className="d-grid gap-1 col-6 mx-auto",)
   

In [16]:
Holiday_dict = {'Holiday':0, 'No Holiday':1}
Holiday_option = [{'label':key, 'value':value} for key, value in Holiday_dict.items()]

Function_dict = {'No':0, 'Yes':1}
Function_option = [{'label':key, 'value':value} for key, value in Function_dict.items()]

In [17]:
# date field
def drawDateTime(id_value):
    return dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Date Picker"),
                
                html.Div([ 
                    dcc.DatePickerSingle(
                        id=id_value,
                        min_date_allowed=date(2017, 8, 5),
                        max_date_allowed=date(2022, 9, 19, ),
                        initial_visible_month=date(2021, 11, 6),
                        date=date(2021, 11, 6),
                        style={'width':'100%'}
                    )]),
                html.Br()

            ])
        ], width=6)
    ], justify="center", align='center')

In [18]:
def drawOutput():
    return dbc.Row([
        dbc.Col([
            html.Div([
                html.Div([
                    html.P(children="Prediction will be appear Here!",
                           id='Output',
                           style={'text-align':'center'})
                ]),
                html.Br(),
            ]),

        ], width=6),
    ], justify="center", align='center')




## Deploy

In [19]:
app = dash.Dash(
            external_stylesheets=[dbc.themes.SLATE],
            update_title='Loading...',
            title='Seoul Bike Rental'
)

server = app.server


# ====================================Start of Design================================= #
app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
# =======================================Header======================================== #
            dbc.Row([
                dbc.Col([
                    drawText()
                ], width=8)
            ], justify="center", align='center'),
            
            html.Br(),
            
# ====================================Input=========================================== #
            dbc.Row([
                dbc.Col([
                    dbc.Card(
                        dbc.CardBody([
                            html.Div([
                                drawDateTime('datetime'),
                                drawInput('Hour', 'Hour(24)'),
                                drawInput('Temperature', 'Temperature(°C)'),
                                drawInput('humidity', 'Humidity(%)'),
                                drawInput('wind_speed', 'Wind speed (m/s)'),
                                drawInput('visability', 'Visibility (10m)'),
                                drawInput('solar', 'Solar Radiation (MJ/m2)'),
                                drawInput('rainfall', 'Rainfall(mm)'),
                                drawlist(Holiday_option, 'Holiday'),
                                drawlist(Function_option, 'Functioning_Day'),
                                drawOutput(),
                                drawButton('Input', 'Predict')
                            ]) 
                        ])
                    ),
                ], width=8),
            ], justify="center", align='center'),

        ]), color = 'dark',
    )
])

# ===============================================Callback method=============================== #
@app.callback(
    Output('Output', 'children'),
    Input('Input', 'n_clicks'),
    State('datetime', 'date'),
    State('Hour', 'value'),
    State('Temperature', 'value'),
    State('humidity', 'value'),
    State('wind_speed', 'value'),
    State('visability', 'value'),
    State('solar', 'value'),
    State('rainfall', 'value'),
    State('Holiday', 'value'),
    State('Functioning_Day', 'value'),
    
)
# ===============================================Update Output=============================== #
def update_output(n_click,date_value, hour, temp, humidity, wind_speed, visiabilty, solar, rain, holiday, Function_Day):
    if n_click == 0:
        output  = "Prediction will be appear Here!"
    else:
        output = input_test(date_value, hour, temp, humidity, wind_speed, visiabilty, solar, rain, holiday, Function_Day)
    return output

# ==========================================Run Code========================================== #
if __name__ == '__main__':
    app.run_server(port=6080)

Dash is running on http://127.0.0.1:6080/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6080/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Nov/2021 18:12:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 18:12:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 18:12:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 18:12:46] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 18:12:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 18:12:46] "GET /assets/favicon.ico?m=1636215119.2524703 HTTP/1.1" 200 -


### Thanks For Read My NoteBook :)